<a href="https://colab.research.google.com/github/enyeneraph/Python-Jobs-Analysis/blob/main/Webscraping_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the necessary libraries

In [1]:
import requests
from html.parser import HTMLParser
import re

In [2]:
class JobDetailsParser(HTMLParser):
  class_list = list()
  
  ''' This class extracts details such as job title, company name, 
  location, category, date posted as well as job description from 
  each page opened. '''

  def __init__(self):
    self.ress = list()
    self.job_data = False
    self.extract_jobdetails = False
    self.company_details = False
    self.extract_jobtitle = False
    self.extract_companyname = False
    self.extract_location = False
    self.extract_category = False
    self.extract_type = False
    self.extract_time = False
    self.extract_urls = False
    self.extract_nextpage = False
    self.page_urls = list()
    super().__init__()


  def handle_starttag(self, tag, attrs):
  #accessing the list of jobs
    if ('class', 'list-recent-jobs list-row-container menu') in attrs:
      self.job_data = True
    elif self.job_data  and tag == 'li':
      self.extract_jobdetails = True
      self.perjob_list = list()

    #jobtitle and company name
    elif self.extract_jobdetails and ('class', 'listing-company-name') in attrs:
      self.company_details = True
    elif tag == 'a' and self.company_details:
      self.extract_jobtitle = True
      self.extract_companyname = True
    #location
    elif ('class', 'listing-location') in attrs:
      self.extract_location = True
    #type
    elif self.extract_jobdetails and ('class', 'listing-job-type') in attrs:
      self.extract_type = True
      self.type_str = ''
    #category
    elif ('class', 'listing-company-category') in attrs:
      self.extract_category = True
    #time
    elif tag == 'time':
      self.extract_time = True
    #nextpage
    elif ('class', 'pagination menu') in attrs:
      self.extract_urls = True
    elif self.extract_urls:
      a = attrs[-1][1]
      self.page_urls.append('https://www.python.org/jobs/' + a)
      # print(self.page_urls)

  def handle_endtag(self, tag):
    #type
    if self.extract_type and tag == 'span':
      self.extract_type = False
    #nextpage
    elif self.extract_urls and tag == 'ul':
      self.extract_urls = False
      self.next_page_url = self.page_urls[-1]

  
  def handle_data(self, data):
    data = data.strip()
    #job title
    if self.extract_jobtitle and data != '':
      self.extract_jobtitle = False
      self.perjob_list.append(data)
      
    #companyname
    elif self.extract_companyname and data != '':
      self.perjob_list.append(data)
      self.extract_companyname = False
      self.company_details = False

    #location
    elif self.extract_location and data != '':
      self.perjob_list.append(data)
      self.extract_location = False

    #type
    elif self.extract_type and data != '':
      self.type_str += data

    #time
    elif self.extract_time and data!= '':
      self.perjob_list.append(data)
      self.extract_time = False
    
    # category
    elif self.extract_category and data != '':
      self.perjob_list.extend((self.type_str, data))
      self.class_list.append(self.perjob_list)
      self.extract_category = False
    
    #nextpage
    elif self.extract_urls and data == 'Next':
      self.extract_nextpage = True



In [3]:
list_of_list = list()

In [4]:
def function_(url):
  '''a recursive fuction that opens a url, calls the JobDetailsParser class,  
  processes jobs, extracts the url for the next page and calls itself  
  with the new next page url'''

  text = requests.get(url).text
  parser = JobDetailsParser()
  parser.feed(text)
  list_of_list = parser.class_list
  next_page_url =  parser.next_page_url
  if  next_page_url != 'https://www.python.org/jobs/':
    function_(next_page_url)
  return list_of_list



In [5]:
#run this only once.
jobs_list = function_('https://www.python.org/jobs/')

In [6]:
len(jobs_list)

151

In [7]:
#converting the lists of jobs and their descriptions  to a dataframe
import pandas as pd
columns = ('Job_Title', 'Company', 'Location', 'Date_Posted', 'Looking_for','Category')
df = pd.DataFrame(jobs_list, columns=columns)
 

In [8]:
df.head()

,Job_Title,Company,Location,Date_Posted,Looking_for,Category
0,Senior Backend Developer,Cashlink,Remote-First (people are around Frankfurt/Berl...,20 November 2021,"Back end,Database,Web, Django, Postgres, Docke...",Developer / Engineer
1,Sr. Python Developer,CGI,"Waterville, Maine, USA",18 November 2021,,Developer / Engineer
2,Python Engineer,DevsData,"Warsaw, Partly Remote, Poland",18 November 2021,"Back end,Cloud, Python, AWS, Docker",Developer / Engineer
3,Backend Engineer (Remote),PolicyFly,"Remote, United States",17 November 2021,"Back end,Big Data,Cloud,Database,Integration, ...",Developer / Engineer
4,Software Developer – Data Engineering,Digital Science,"Europe, Europe",17 November 2021,"Big Data,Cloud,Database,Machine Learning,Text ...",Developer / Engineer


In [9]:
#converting from DataFrame to csv
from google.colab import files
df.to_csv('pythonjobs.csv', index=False) 
files.download('pythonjobs.csv')
